In [1]:
import numpy as np

In [6]:
def allHRU(bare_input, forest_input, grass_input, rip_input, 
           bare_storage, forest_storage, grass_storage, rip_storage,
           bare_parameters, forest_parameters, grass_parameters, rip_parameters,
           Slowstorage, slow_parameters):
    #this function runs thy model for different HRUs
    #bare rock HRU
    bare_outflow, bare_storag, Bare_precipitation, Bare_Storages = hillslopeHRU(bare_input, bare_storage, bare_parameters)
    # forest HRU
    forest_outflow, forest_storage, Forest_precipitation, Forest_Stora0ges = hillslopeHRU(forest_input, forest_storage, forest_parameters)
    # Grassland HRU
    grass_outflow, grass_storage, Grass_precipitation, Grass_Storages= hillslopeHRU(grass_input, grass_storage, grass_parameters)
    # riparian HRU
    rip_outflow, rip_storage, Rip_precipitation, Rip_Storages= riparianHRU(rip_input, rip_storage, rip_parameters)
    # total flow into groundwater is the weighted sum of the HRUs according to areal extent (this was already done in hillslopeHRU)
    Total_GWflow = bare_outflow.GWflow + forest_outflow.GWflow  + grass_outflow.GWflow
    # Groundwater storage
    Riparian_Discharge, Slow_Discharge, Slowstorage_New = slowstorage(Total_GWflow, Slowstorage, rip_input.Area_HRU, slow_parameters.Ks, slow_parameters.Ratio_Riparian)
    #return all storage values, all evaporation values, Fast_Discharge and Slow_Discharge
    # calculate total discharge of the timestep using weighted sum of each HRU
    Total_Discharge = bare_outflow.Fast_Discharge  + forest_outflow.Fast_Discharge + grass_outflow.Fast_Discharge  + rip_outflow.Fast_Discharge  + Slow_Discharge
    Total_Soil_Evaporation = bare_outflow.Soil_Evaporation + forest_outflow.Soil_Evaporation + grass_outflow.Soil_Evaporation  + rip_outflow.Soil_Evaporation
    Total_Interception_Evaporation = bare_outflow.Interception_Evaporation  + forest_outflow.Interception_Evaporation + grass_outflow.Interception_Evaporation + rip_outflow.Interception_Evaporation
    assert Riparian_Discharge >= 0
    assert Total_Discharge >= 0
    assert Total_Interception_Evaporation >= 0
    assert Total_Soil_Evaporation >= 0
    assert Slowstorage_New >= 0
    Total_Flows = Total_Discharge + Total_Soil_Evaporation + Total_Interception_Evaporation + Riparian_Discharge
    Total_Storages = Bare_Storages * bare_input.Area_HRU + Forest_Storages * forest_input.Area_HRU + Grass_Storages * grass_input.Area_HRU + Rip_Storages * rip_input.Area_HRU + Slowstorage_New - Slowstorage
    Precipitation = Bare_precipitation * bare_input.Area_HRU + Forest_precipitation * forest_input.Area_HRU + Grass_precipitation * grass_input.Area_HRU + Rip_precipitation * rip_input.Area_HRU
    #@assert -0.00000001 <= Precipitation + rip_input.Riparian_Discharge - (Total_Flows + Total_Storages) <= 0.00000001
    Waterbalance = Precipitation + rip_input.Riparian_Discharge - (Total_Flows + Total_Storages)
    
    return Riparian_Discharge, Total_Discharge, Total_Interception_Evaporation, Total_Soil_Evaporation, bare_storage, forest_storage, grass_storage, rip_storage, Slowstorage_New, Waterbalance, Precipitation


In [18]:
def input_timestep(Input, Evaporation_Mean, Precipitation, Temperature):
    
    #Input.Potential_Evaporation::Array{Float64,1} = Evaporation
    Input.Potential_Evaporation_Mean = Evaporation_Mean
    # get the precipitation data of the necessary elevations
    Precipitation_HRU = []
    Temperature_HRU = []
    for i in Input.Elevation_Count:
        Precipitation_HRU.append(Precipitation[i])
        Temperature_HRU.append(Temperature[i])
    Input.Precipitation = Precipitation_HRU
    Input.Temp_Elevation = Temperature_HRU
    
    return Input

In [19]:
def run_model(Area, Evaporation_Mean, Precipitation, Temp,
                bare_input, forest_input, grass_input, rip_input,
                bare_storage, forest_storage, grass_storage, rip_storage, Slowstorage,
                bare_parameters, forest_parameters, grass_parameters, rip_parameters, slow_parameters,
                Total_Elevationbands, Elevation_Percentage):
     # the function takes as input the parameters of each HRU, the inital storage values of each HRU, the inital value of the slow storage
    # KS, ratio riparian, all inputs
    
    #define maximum time
    tmax = len(Precipitation[:,1])
    
    Discharge = np.zeros(tmax)
    WBtotal = np.zeros(tmax)
    Snow_Extend = np.zeros(tmax, Total_Elevationbands)
    
    # store the initial storage values, Does not take into account GW storage!!
    # Assumption_ GW storage is 0 at start
    Initial_Storage_bare = bare_storage.Fast + sum(bare_storage.Interception) + sum(bare_storage.Snow) + bare_storage.Soil
    Initial_Storage_forest = forest_storage.Fast + sum(forest_storage.Interception) + sum(forest_storage.Snow) + forest_storage.Soil
    Initial_Storage_grass = grass_storage.Fast + sum(grass_storage.Interception) + sum(grass_storage.Snow) + grass_storage.Soil
    Initial_Storage_rip = rip_storage.Fast + sum(rip_storage.Interception) + sum(rip_storage.Snow) + rip_storage.Soil
    Initial_Storage = Initial_Storage_bare + Initial_Storage_forest + Initial_Storage_grass + Initial_Storage_rip + Slowstorage
    
    for t in range(0, tmax+1):
        # at each timestep new temp, precipitation and Epot values have to be delivered
        # areas don't change
        # riparian discharge from former timestep has to be used
        # gives the current precipitation, evaporation and temperature
        Evaporation_Mean_Current = Evaporation_Mean[t]
        Precipitation_Current = Precipitation[t, :]
        Temperature_Current = Temp[t, :] 
        
        bare_input = input_timestep(bare_input, Evaporation_Mean_Current, Precipitation_Current, Temperature_Current)
        forest_input = input_timestep(forest_input, Evaporation_Mean_Current, Precipitation_Current, Temperature_Current)
        grass_input = input_timestep(grass_input, Evaporation_Mean_Current, Precipitation_Current, Temperature_Current)
        rip_input = input_timestep(rip_input, Evaporation_Mean_Current, Precipitation_Current, Temperature_Current)
        
        # give new riparian discharge as input for next timestep
        rip_input.Riparian_Discharge = Riparian_Discharge
        # save the fluxes of the current timestep
        Discharge[t] = Total_Discharge/1000 * Area / (3600 * 24) # mm convert it to meter and than * area / seconds in one day
        # OPTIONAL: store the total amount of snow in each elevation (considering whole catchment)
        Snow_Elevations, Snow_Extend[t,:] = snowperelevation(bare_input, forest_input, grass_input, rip_input, bare_storage, forest_storage, grass_storage, rip_storage, Total_Elevationbands)
        #Bare_Snow[t,:]::Array{Float64,1} = bare_storage.Snow
        Snow_Extend[t,:] = Snow_Extend[t,:] / Elevation_Percentage

        WBtotal[t] = WB
                
    # Check Water Balance
    Waterbalance = sum(WBtotal)
    assert Waterbalance <= 10**(-10)
    
    return Discharge, Snow_Extend, Waterbalance


In [13]:
def runmodel_alloutput(Area, Evaporation_Mean, Precipitation, Temp,
                bare_input, forest_input, grass_input, rip_input,
                bare_storage, forest_storage, grass_storage, rip_storage, Slowstorage,
                bare_parameters, forest_parameters, grass_parameters, rip_parameters, slow_parameters, Total_Elevationbands, Elevation_Percentage):
    # the function takes as input the parameters of each HRU, the inital storage values of each HRU, the inital value of the slow storage
    # KS, ratio riparian, all inputs

    # define the maximum time
    tmax = len(Precipitation[:,0])

    # make arrays for each Model Component
    Int_Evaporation = np.zeros(tmax) #interception evaporation
    Soil_Evaporation = np.zeros(tmax) #soil evaporation
    Discharge = np.zeros(tmax)

    # store the initial storage values, Does not take into account GW storage!!
    # Assumption_ GW storage is 0 at start
    #TO DO: average values over area
    # as form now only works if storage input is 0 at start
    Initial_Storage_bare = bare_storage.Fast + sum(bare_storage.Interception) + sum(bare_storage.Snow) + bare_storage.Soil
    Initial_Storage_forest = forest_storage.Fast + sum(forest_storage.Interception) + sum(forest_storage.Snow) + forest_storage.Soil
    Initial_Storage_grass = grass_storage.Fast + sum(grass_storage.Interception) + sum(grass_storage.Snow) + grass_storage.Soil
    Initial_Storage_rip = rip_storage.Fast + sum(rip_storage.Interception) + sum(rip_storage.Snow) + rip_storage.Soil
    Initial_Storage = Initial_Storage_bare + Initial_Storage_forest + Initial_Storage_grass + Initial_Storage_rip + Slowstorage

    #OPTIONAL: store all storage states
    Interceptionstorage = np.zeros(tmax, 4) #storage interception
    Snowstorage = np.zeros(tmax)
    Soilstorage = np.zeros(tmax, 4) #stroage unsaturated zone
    Faststorage = np.zeros(tmax, 4) #storage fast
    GWstorage = np.zeros(tmax) #storage GW
    WBtotal = np.zeros(tmax)
    Snow_Extend = np.zeros(tmax, Total_Elevationbands)
    Precipitation_Total = np.zeros(tmax)
    Snow_Elevations = np.zeros(tmax, Total_Elevationbands)
    Bare_Snow = np.zeros(tmax, bare_input.Nr_Elevationbands)

    for t in range(tmax):
        # at each timestep new temp, precipitation and Epot values have to be delivered
        # areas don't change
        # riparian discharge from former timestep has to be used
        # gives the current precipitation, evaporation and temperature
        Evaporation_Mean_Current = Evaporation_Mean[t]
        Precipitation_Current = Precipitation[t, :]
        Temperature_Current = Temp[t, :]

        bare_input = input_timestep(bare_input, Evaporation_Mean_Current, Precipitation_Current, Temperature_Current)
        forest_input = input_timestep(forest_input, Evaporation_Mean_Current, Precipitation_Current, Temperature_Current)
        grass_input = input_timestep(grass_input, Evaporation_Mean_Current, Precipitation_Current, Temperature_Current)
        rip_input = input_timestep(rip_input, Evaporation_Mean_Current, Precipitation_Current, Temperature_Current)

        Riparian_Discharge, Total_Discharge, Total_Interception_Evaporation, Total_Soil_Evaporation, bare_storage, forest_storage, grass_storage, rip_storage, Slowstorage, WB, Total_Prec = allHRU(bare_input, forest_input, grass_input, rip_input,
                                                                                                            bare_storage, forest_storage, grass_storage, rip_storage,
                                                                                                            bare_parameters, forest_parameters, grass_parameters, rip_parameters,
                                                                                                            Slowstorage, slow_parameters)
        # give new riparian discharge as input for next timestep
        rip_input.Riparian_Discharge = Riparian_Discharge
        # save the fluxes of the current timestep
        Discharge[t] = Total_Discharge/1000 * Area / (3600 * 24) # mm convert it to meter and than * area / seconds in one day
        Int_Evaporation[t] = Total_Interception_Evaporation
        Soil_Evaporation[t] = Total_Soil_Evaporation
        #OPTIONAL: store all storage states at each timestep
        #get the total value stored as mean value of elevations and areal extent of HRU
        Bare_Interceptionstorage, Bare_Snowstorage = Storage_Total(bare_storage, bare_input)
        Forest_Interceptionstorage, Forest_Snowstorage = Storage_Total(forest_storage, forest_input)
        Grass_Interceptionstorage, Grass_Snowstorage = Storage_Total(grass_storage, grass_input)
        Rip_Interceptionstorage, Rip_Snowstorage = Storage_Total(rip_storage, rip_input)

        # OPTIONAL: store the total amount of snow in each elevation (considering whole catchment)
        Snow_Elevations[t, :], Snow_Extend[t,:] = snowperelevation(bare_input, forest_input, grass_input, rip_input, bare_storage, forest_storage, grass_storage, rip_storage, Total_Elevationbands)
        #Bare_Snow[t,:] = bare_storage.Snow
        Snow_Extend[t,:] = Snow_Extend[t,:] / Elevation_Percentage
        Snow_Elevations[t,:] = Snow_Elevations[t,:] /Elevation_Percentage

        Interceptionstorage[t, :] = [Bare_Interceptionstorage, Forest_Interceptionstorage, Grass_Interceptionstorage, Rip_Interceptionstorage]
        Snowstorage[t] = Bare_Snowstorage * bare_input.Area_HRU + Forest_Snowstorage * forest_input.Area_HRU + Grass_Snowstorage * grass_input.Area_HRU + Rip_Snowstorage * rip_input.Area_HRU
        Soilstorage[t, :] = [bare_storage.Soil, forest_storage.Soil, grass_storage.Soil, rip_storage.Soil]
        Faststorage[t, :] = [bare_storage.Fast, forest_storage.Fast, grass_storage.Fast, rip_storage.Fast]
        GWstorage[t] = Slowstorage
        WBtotal[t] = WB
        Precipitation_Total[t] = Total_Prec
 
    # Check Water Balance
    # calculate the water balance at each timestep and sum it at the end for getting waterbalance over all timesteps
    Waterbalance = sum(WBtotal)
    assert Waterbalance <= 10**(-10)
    return Discharge, Snow_Extend, GWstorage, Snowstorage, Snow_Elevations, Soilstorage, Faststorage #, Interceptionstorage::Array{Float64,2},


In [14]:
def Storage_Total(Storage, Input):
    #print([bare_storage.Interception[1] * bare_input.Area_Elevations[1], forest_storage.Interception[1] * forest_input.Area_Elevations[1] , grass_storage.Interception[1] * grass_input.Area_Elevations[1], rip_storage.Interception[1] * rip_input.Area_Elevations[1]])
    # this function gives the total storage stored in reservoirs that are elevation distributed
    Total_Interception_Storage = 0
    Total_Snow_Storage = 0
    for i in range(Input.Nr_Elevationbands):
        Interception_Storage = Storage.Interception[i] * Input.Area_Elevations[i]
        Snow_Storage = Storage.Snow[i] * Input.Area_Elevations[i]
        Former_Total_Interception_Storage = Total_Interception_Storage
        Former_Total_Snow_Storage = Total_Snow_Storage

        Total_Interception_Storage += Interception_Storage
        Total_Snow_Storage += Snow_Storage

        assert Total_Interception_Storage >= Interception_Storage
        assert Total_Snow_Storage >= Snow_Storage
        assert Total_Interception_Storage >= Former_Total_Interception_Storage
        assert Total_Snow_Storage >= Former_Total_Snow_Storage
    
    return Total_Interception_Storage, Total_Snow_Storage


In [17]:
def snowperelevation(Bare, Forest, Grass, Rip, Bare_Storage, Forest_Storage, Grass_Storage, Rip_Storage, Total_Elevationbands):
    Snowstorage = np.zeros(Total_Elevationbands)
    Snow_Cover = np.zeros(Total_Elevationbands)
    bare_count = 1
    forest_count = 1
    grass_count = 1
    rip_count = 1
    for i in range(Total_Elevationbands):
        if bare_count <= len(Bare.Elevation_Count) & Bare.Elevation_Count[bare_count] == i:
            Snowstorage[i]+= Bare_Storage.Snow[bare_count] * Bare.Area_Elevations[bare_count] * Bare.Area_HRU
            Snow_Cover[i]+= Bare_Storage.Snow_Cover[bare_count] * Bare.Area_Elevations[bare_count] * Bare.Area_HRU
            bare_count+= 1
       
        if forest_count <= len(Forest.Elevation_Count) & Forest.Elevation_Count[forest_count] == i:
            Snowstorage[i]+= Forest_Storage.Snow[forest_count] * Forest.Area_Elevations[forest_count] * Forest.Area_HRU
            Snow_Cover[i]+= Forest_Storage.Snow_Cover[forest_count] * Forest.Area_Elevations[forest_count] * Forest.Area_HRU
            forest_count+= 1
        
        if grass_count <= len(Grass.Elevation_Count) & Grass.Elevation_Count[grass_count] == i:
            Snowstorage[i]+= Grass_Storage.Snow[grass_count] * Grass.Area_Elevations[grass_count] * Grass.Area_HRU
            Snow_Cover[i]+= Grass_Storage.Snow_Cover[grass_count] * Grass.Area_Elevations[grass_count] * Grass.Area_HRU
            grass_count += 1
        
        if rip_count <= len(Rip.Elevation_Count) & Rip.Elevation_Count[rip_count] == i:
            Snowstorage[i]+= Rip_Storage.Snow[rip_count] * Rip.Area_Elevations[rip_count] * Rip.Area_HRU
            Snow_Cover[i]+= Rip_Storage.Snow_Cover[rip_count] * Rip.Area_Elevations[rip_count] * Rip.Area_HRU
            rip_count += 1
    
    return Snowstorage, Snow_Cover
